<a href="https://colab.research.google.com/github/haidarhakim/Belajar-Asah/blob/main/Proyek_Sentimen_Adhan_Haidar_Hakim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test


In [3]:
pip install google-play-scraper pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00


In [1]:
import pandas as pd
from google_play_scraper import Sort, reviews

# Mengambil ulasan dari aplikasi Honor of Kings
result, continuation_token = reviews(
    'com.levelinfinite.sgameGlobal',
    lang='id',                # Bahasa ulasan (Indonesia)
    country='id',             # Negara (Indonesia)
    sort=Sort.NEWEST,         # Mengurutkan berdasarkan ulasan terbaru
    count=20000,              # Jumlah ulasan yang ingin diambil
    filter_score_with=None    # Mengambil semua skor rating
)

# Membuat DataFrame dari hasil scraping
df = pd.DataFrame(result)

# Menyimpan DataFrame ke dalam file CSV
df.to_csv('honor_of_kings_reviews.csv', index=False)

print("Scraping selesai. Data tersimpan dalam file 'honor_of_kings_reviews.csv'")

Scraping selesai. Data tersimpan dalam file 'honor_of_kings_reviews.csv'


Pelatihan Model


In [1]:
pip install pandas numpy scikit-learn nltk sastrawi gensim tensorflow

In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# Muat dataset
df = pd.read_csv('honor_of_kings_reviews.csv')

# Menampilkan 5 baris pertama data
print(df.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                               reviewId         userName  \
0  76cc1fd7-e7bb-4f2f-a530-8e2f2c0d15ff  Pengguna Google   
1  f975780d-55c8-442f-bc3a-30424d026371  Pengguna Google   
2  da802bdc-34b9-4066-8149-4ae5de585640  Pengguna Google   
3  c2fb643f-88db-4bc6-a239-116097c1ea67  Pengguna Google   
4  aa8344d9-cd6f-49cc-be45-59e4a6dc8fb1  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0                                       gemnya haram      1              0   
1                  sinyal full main tp mesti nge lag      1              0   
2                              game buruk mending ml      1         

In [3]:
# Fungsi untuk membersihkan teks
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()  # Mengubah teks menjadi huruf kecil
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # Menghapus karakter selain huruf dan spasi
        return text
    return ""

# Membersihkan kolom 'content'
df['cleaned_content'] = df['content'].apply(clean_text)

# Fungsi untuk pelabelan sentimen
def label_sentiment(score):
    if score >= 4:
        return 'positif'
    elif score == 3:
        return 'netral'
    else:
        return 'negatif'

# Membuat kolom sentimen
df['sentiment'] = df['score'].apply(label_sentiment)

# Menghapus baris dengan teks kosong setelah pembersihan
df = df[df['cleaned_content'] != '']

# Menampilkan distribusi sentimen
print(df['sentiment'].value_counts())

sentiment
positif    12198
negatif     6387
netral      1224
Name: count, dtype: int64


In [4]:
#Ekstraksi fitur dan pembagian data

# Inisialisasi stop words Bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

# Ekstraksi Fitur dengan TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=list(stop_words))
X_tfidf = tfidf_vectorizer.fit_transform(df['cleaned_content'])

# Memisahkan fitur (X) dan label (y)
y = df['sentiment']

# Pembagian data (akan disesuaikan untuk setiap skema)

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


# Skema 1: SVM, TF-IDF 70/30

In [8]:

# Pembagian data 80/20
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Pelatihan model SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_1, y_train_1)

# Prediksi pada data uji
y_pred_1 = svm_model.predict(X_test_1)

# Evaluasi model
accuracy_1 = accuracy_score(y_test_1, y_pred_1)
print(f"Akurasi Skema 1 (SVM, TF-IDF, 80/20): {accuracy_1:.2f}")
print(classification_report(y_test_1, y_pred_1))


Akurasi Skema 1 (SVM, TF-IDF, 70/30): 0.80
              precision    recall  f1-score   support

     negatif       0.73      0.75      0.74      1927
      netral       0.00      0.00      0.00       354
     positif       0.83      0.90      0.87      3662

    accuracy                           0.80      5943
   macro avg       0.52      0.55      0.53      5943
weighted avg       0.75      0.80      0.77      5943



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Skema 2: Random FOrest, Word2Vec, 80/20

In [9]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Pelatihan model Random Forest
rf_model_1 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_1.fit(X_train_2, y_train_2)

# Prediksi pada data uji
y_pred_2 = rf_model_1.predict(X_test_2)

# Evaluasi model
accuracy_2 = accuracy_score(y_test_2, y_pred_2)
print(f"Akurasi Skema 2 (RF, TF-IDF, 80/20): {accuracy_2:.2f}")
print(classification_report(y_test_2, y_pred_2))


Akurasi Skema 2 (RF, TF-IDF, 80/20): 0.79
              precision    recall  f1-score   support

     negatif       0.71      0.75      0.73      1283
      netral       0.14      0.00      0.01       250
     positif       0.83      0.89      0.86      2429

    accuracy                           0.79      3962
   macro avg       0.56      0.55      0.53      3962
weighted avg       0.75      0.79      0.76      3962



Skema 3: Random Forest, TF-IDF 80/20


In [10]:

# Pembagian data 70/30
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Pelatihan model Random Forest
rf_model_2 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_2.fit(X_train_3, y_train_3)

# Prediksi pada data uji
y_pred_3 = rf_model_2.predict(X_test_3)

# Evaluasi model
accuracy_3 = accuracy_score(y_test_3, y_pred_3)
print(f"Akurasi Skema 3 (RF, TF-IDF, 70/30): {accuracy_3:.2f}")
print(classification_report(y_test_3, y_pred_3))

Akurasi Skema 3 (RF, TF-IDF, 70/30): 0.79
              precision    recall  f1-score   support

     negatif       0.71      0.75      0.73      1283
      netral       0.14      0.00      0.01       250
     positif       0.83      0.89      0.86      2429

    accuracy                           0.79      3962
   macro avg       0.56      0.55      0.53      3962
weighted avg       0.75      0.79      0.76      3962



In [13]:
# Contoh ulasan baru
new_review = "Game sampah!"

# Preprocessing ulasan baru
cleaned_review = clean_text(new_review)

# Transformasi dengan TF-IDF Vectorizer yang sudah dilatih
vectorized_review = tfidf_vectorizer.transform([cleaned_review])

# Prediksi sentimen menggunakan model terbaik (misalnya SVM)
predicted_sentiment = svm_model.predict(vectorized_review)

print(f"Ulasan: '{new_review}'")
print(f"Prediksi Sentimen: {predicted_sentiment[0]}")

Ulasan: 'Game sampah!'
Prediksi Sentimen: negatif


In [ ]:
#ini mau nyoba 1 doang dah sek..
import pandas as pd
import re
from google_play_scraper import Sort, reviews
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 1. Scraping Data
result, _ = reviews(
    'com.levelinfinite.sgameGlobal',
    lang='id',
    country='id',
    sort=Sort.NEWEST,
    count=700000
)
df = pd.DataFrame(result)

# 2. Preprocessing dan Pelabelan
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        return text
    return ""

def label_sentiment(score):
    if score >= 4:
        return 'positif'
    elif score == 3:
        return 'netral'
    else:
        return 'negatif'

df['cleaned_content'] = df['content'].apply(clean_text)
df['sentiment'] = df['score'].apply(label_sentiment)
df = df[df['cleaned_content'] != '']

# 3. Ekstraksi Fitur dan Pembagian Data
tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(df['cleaned_content'])
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Pelatihan Model
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# 5. Evaluasi
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model: {accuracy:.2f}")

# 6. Inference
new_review = "Gamenya jelek, sering nge-lag."
cleaned_review = clean_text(new_review)
vectorized_review = tfidf.transform([cleaned_review])
predicted_sentiment = model.predict(vectorized_review)
print(f"Prediksi untuk '{new_review}': {predicted_sentiment[0]}")

#ini terpaksa ga tak jalanin karna 1 jam setengah, aganya nyesel bang nyoba: